<a href="https://colab.research.google.com/github/shelke16/HPC-Mini-Project/blob/main/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving students.csv to students (1).csv


In [ ]:
import pandas as pd
import concurrent.futures
import time

# Load CSV without header and set column names
df = pd.read_csv('students.csv', header=None)
df.columns = ['ID', 'Name', 'Age', 'Score'] # Set column names here

# Convert 'Score' column to numeric, handling errors
df['Score'] = pd.to_numeric(df['Score'], errors='coerce')

# Initialize shared variables
high_scorers = []
failed_students = []
score_sum = 0
highest_score = 0
topper_name = ""

# Function to process each student record
def process_student(row):
    global highest_score, topper_name
    info = {}
    score = row['Score'] # Now 'Score' column should exist

    if score >= 80:
        info['high'] = row
    if score < 40:
        info['fail'] = row
    info['score'] = score
    return info

# Start timing
start_time = time.time()

results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_student, [row for _, row in df.iterrows()]))

total_score = 0
for res in results:
    total_score += res['score']
    if 'high' in res:
        high_scorers.append(res['high'])
    if 'fail' in res:
        failed_students.append(res['fail'])
    if res['score'] > highest_score:
        highest_score = res['score']
        topper_name = res.get('high', {}).get('Name', "Unknown")

# Display results
print("Students with score >= 80:")
print("-" * 40)
for student in high_scorers:
    print(f"ID: {student['ID']} | Name: {student['Name']} | Age: {student['Age']} | Score: {student['Score']}")

print("\nSummary")
print("-" * 40)
print(f"Total Students: {len(df)}")
print(f"High Scorers (>=80): {len(high_scorers)}")
print(f"Failed Students (<40): {len(failed_students)}")
print(f"Average Score: {total_score / len(df):.2f}")
print(f"Topper: {topper_name} with score {highest_score}")
print(f"\nExecution Time: {time.time() - start_time:.6f} seconds")

Students with score >= 80:
----------------------------------------
ID: 1 | Name: Alice | Age: 20 | Score: 85
ID: 3 | Name: Charlie | Age: 22 | Score: 90
ID: 5 | Name: Eva | Age: 21 | Score: 95
ID: 6 | Name: Farhan | Age: 23 | Score: 88
ID: 8 | Name: Hari | Age: 22 | Score: 80
ID: 9 | Name: Isha | Age: 21 | Score: 92
ID: 11 | Name: Kim | Age: 20 | Score: 83
ID: 12 | Name: Liam | Age: 22 | Score: 89
ID: 14 | Name: Nina | Age: 20 | Score: 86
ID: 15 | Name: Om | Age: 23 | Score: 91
ID: 17 | Name: Quinn | Age: 21 | Score: 84
ID: 20 | Name: Tina | Age: 19 | Score: 94
ID: 21 | Name: Uday | Age: 21 | Score: 81
ID: 22 | Name: Veena | Age: 20 | Score: 88
ID: 24 | Name: Xena | Age: 21 | Score: 85
ID: 26 | Name: Zoya | Age: 20 | Score: 87

Summary
----------------------------------------
Total Students: 26
High Scorers (>=80): 16
Failed Students (<40): 1
Average Score: 80.04
Topper: Eva with score 95

Execution Time: 0.006500 seconds


In [ ]:
import pandas as pd
import time

# ✅ Step 1: Read the CSV file and assign column names
df = pd.read_csv("students.csv", names=["ID", "Name", "Age", "Score"])
print("✅ Columns in the CSV:", df.columns.tolist())  # Optional: just to verify

# ✅ Step 2: Initialize values
start_time = time.time()

total_students = len(df)
high_scorers_df = df[df["Score"] >= 80]
failed_students_df = df[df["Score"] < 40]
average_score = df["Score"].mean()
topper = df.loc[df["Score"].idxmax()]

# ✅ Step 3: Output results
print("\nStudents with score >= 80:")
print("----------------------------------")
for _, row in high_scorers_df.iterrows():
    print(f"ID: {row['ID']} | Name: {row['Name']} | Age: {row['Age']} | Score: {row['Score']}")

print("\nExecution Time: {:.6f} seconds".format(time.time() - start_time))

print("\n----------------------------------------")
print(f"Total Students: {total_students}")
print(f"High Scorers (>=80): {len(high_scorers_df)}")
print(f"Failed Students (<40): {len(failed_students_df)}")
print(f"Average Score: {average_score:.2f}")
print(f"Topper: {topper['Name']} with score {topper['Score']}")


✅ Columns in the CSV: ['ID', 'Name', 'Age', 'Score']

Students with score >= 80:
----------------------------------
ID: 1 | Name: Alice | Age: 20 | Score: 85
ID: 3 | Name: Charlie | Age: 22 | Score: 90
ID: 5 | Name: Eva | Age: 21 | Score: 95
ID: 6 | Name: Farhan | Age: 23 | Score: 88
ID: 8 | Name: Hari | Age: 22 | Score: 80
ID: 9 | Name: Isha | Age: 21 | Score: 92
ID: 11 | Name: Kim | Age: 20 | Score: 83
ID: 12 | Name: Liam | Age: 22 | Score: 89
ID: 14 | Name: Nina | Age: 20 | Score: 86
ID: 15 | Name: Om | Age: 23 | Score: 91
ID: 17 | Name: Quinn | Age: 21 | Score: 84
ID: 20 | Name: Tina | Age: 19 | Score: 94
ID: 21 | Name: Uday | Age: 21 | Score: 81
ID: 22 | Name: Veena | Age: 20 | Score: 88
ID: 24 | Name: Xena | Age: 21 | Score: 85
ID: 26 | Name: Zoya | Age: 20 | Score: 87

Execution Time: 0.020206 seconds

----------------------------------------
Total Students: 26
High Scorers (>=80): 16
Failed Students (<40): 1
Average Score: 80.04
Topper: Eva with score 95


In [ ]:
import pandas as pd
from multiprocessing import Pool, cpu_count
import time

# Load the CSV
def load_data(filename):
    try:
        df = pd.read_csv(filename)
        print("Data loaded successfully.")
        return df
    except Exception as e:
        print("Error loading CSV:", e)
        return None

# Function to check if a student is a high scorer
def is_high_scorer(row):
    return row['Score'] >= 80

# Function to check if a student failed
def is_failed(row):
    return row['Score'] < 40

def parallel_filter(df, func):
    with Pool(cpu_count()) as pool:
        results = pool.map(func, [row for _, row in df.iterrows()])
    return df[results]

# Main execution
if __name__ == '__main__':
    start_time = time.time()
    df = load_data("students.csv")

    if df is not None and 'Score' in df.columns:
        total_students = len(df)

        # Parallel filtering
        high_scorers_df = parallel_filter(df, is_high_scorer)
        failed_df = parallel_filter(df, is_failed)

        average_score = df["Score"].mean()
        top_scorer = df.loc[df["Score"].idxmax()]

        # Output
        print("\nStudents with Score >= 80:")
        print("----------------------------------")
        print(high_scorers_df.to_string(index=False))

        print("\n----------------------------------------")
        print(f"Total Students: {total_students}")
        print(f"High Scorers (>=80): {len(high_scorers_df)}")
        print(f"Failed Students (<40): {len(failed_df)}")
        print(f"Average Score: {average_score:.2f}")
        print(f"Topper: {top_scorer['Name']} with score {top_scorer['Score']}")
        print(f"\nExecution Time: {time.time() - start_time:.4f} seconds")
    else:
        print("CSV must contain a 'Score' column.")


Data loaded successfully.
CSV must contain a 'Score' column.


In [ ]:
!pip install dask[complete]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00


In [ ]:
import dask.dataframe as dd

# Load CSV with Dask
df = dd.read_csv("students.csv", names=["ID", "Name", "Age", "Score"], skiprows=1)

# Filter high scorers (Score >= 80)
high_scorers_df = df[df["Score"] >= 80]

# Filter failed students (Score < 40)
failed_students_df = df[df["Score"] < 40]

# Total students
total_students = df.shape[0].compute()

# Number of high scorers and failed students
num_high_scorers = high_scorers_df.shape[0].compute()
num_failed = failed_students_df.shape[0].compute()

# Average score
average_score = df["Score"].mean().compute()

# Top scorer
topper = df.nlargest(1, "Score").compute().iloc[0]

# Show high scorers
print("\nStudents with Score >= 80:")
print("----------------------------------")
print(high_scorers_df.compute().to_string(index=False))

# Summary
print("\n----------------------------------------")
print(f"Total Students: {total_students}")
print(f"High Scorers (>=80): {num_high_scorers}")
print(f"Failed Students (<40): {num_failed}")
print(f"Average Score: {average_score:.2f}")
print(f"Topper: {topper['Name']} with score {topper['Score']}")



Students with Score >= 80:
----------------------------------
 ID    Name  Age  Score
  3 Charlie   22     90
  5     Eva   21     95
  6  Farhan   23     88
  8    Hari   22     80
  9    Isha   21     92
 11     Kim   20     83
 12    Liam   22     89
 14    Nina   20     86
 15      Om   23     91
 17   Quinn   21     84
 20    Tina   19     94
 21    Uday   21     81
 22   Veena   20     88
 24    Xena   21     85
 26    Zoya   20     87

----------------------------------------
Total Students: 25
High Scorers (>=80): 15
Failed Students (<40): 1
Average Score: 79.84
Topper: Eva with score 95


In [ ]:
import pandas as pd
from multiprocessing import Pool, cpu_count
import time

# Load the CSV
def load_data(filename):
    try:
        df = pd.read_csv(filename)
        print("Data loaded successfully.")
        return df
    except Exception as e:
        print("Error loading CSV:", e)
        return None

# Function to check if a student is a high scorer
def is_high_scorer(row):
    return row['Score'] >= 80

# Function to check if a student failed
def is_failed(row):
    return row['Score'] < 40

def parallel_filter(df, func):
    with Pool(cpu_count()) as pool:
        results = pool.map(func, [row for _, row in df.iterrows()])
    return df[results]

# Main execution
if __name__ == '__main__':
    start_time = time.time()
    df = load_data("students.csv")

    if df is not None and 'Score' in df.columns:
        total_students = len(df)

        # Parallel filtering
        high_scorers_df = parallel_filter(df, is_high_scorer)
        failed_df = parallel_filter(df, is_failed)

        average_score = df["Score"].mean()
        top_scorer = df.loc[df["Score"].idxmax()]

        # Output
        print("\nStudents with Score >= 80:")
        print("----------------------------------")
        print(high_scorers_df.to_string(index=False))

        print("\n----------------------------------------")
        print(f"Total Students: {total_students}")
        print(f"High Scorers (>=80): {len(high_scorers_df)}")
        print(f"Failed Students (<40): {len(failed_df)}")
        print(f"Average Score: {average_score:.2f}")
        print(f"Topper: {top_scorer['Name']} with score {top_scorer['Score']}")
        print(f"\nExecution Time: {time.time() - start_time:.4f} seconds")
    else:
        print("CSV must contain a 'Score' column.")


Data loaded successfully.
CSV must contain a 'Score' column.


In [ ]:
import pandas as pd
import multiprocessing
import time

# Load the CSV
df = pd.read_csv("students.csv", names=["ID", "Name", "Age", "Score"], skiprows=1)

# Function to process a chunk of the DataFrame
def process_chunk(chunk):
    high = chunk[chunk['Score'] >= 80]
    failed = chunk[chunk['Score'] < 40]
    top_score = chunk['Score'].max()
    top_row = chunk[chunk['Score'] == top_score].iloc[0]
    return {
        "high": high,
        "failed": failed,
        "sum": chunk['Score'].sum(),
        "count": len(chunk),
        "top": top_row
    }

# Split the DataFrame into chunks for each core
def parallel_process(df):
    num_processes = multiprocessing.cpu_count()
    chunks = np.array_split(df, num_processes)

    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(process_chunk, chunks)

    # Merge results
    high_df = pd.concat([res['high'] for res in results])
    failed_df = pd.concat([res['failed'] for res in results])
    total_sum = sum(res['sum'] for res in results)
    total_count = sum(res['count'] for res in results)

    # Get global topper
    top_student = max(results, key=lambda x: x['top']['Score'])['top']

    return high_df, failed_df, total_sum, total_count, top_student

# Start timing
start_time = time.time()

# Run the parallel processing
import numpy as np
high_df, failed_df, total_sum, total_count, top_student = parallel_process(df)

# Output results
print("\nStudents with Score >= 80:")
print("----------------------------------")
print(high_df.to_string(index=False))

print("\n----------------------------------------")
print(f"Total Students: {total_count}")
print(f"High Scorers (>=80): {len(high_df)}")
print(f"Failed Students (<40): {len(failed_df)}")
print(f"Average Score: {total_sum / total_count:.2f}")
print(f"Topper: {top_student['Name']} with score {top_student['Score']}")

# End timing
end_time = time.time()
print(f"\nExecution Time: {end_time - start_time:.6f} seconds")


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



Students with Score >= 80:
----------------------------------
 ID    Name  Age  Score
  3 Charlie   22     90
  5     Eva   21     95
  6  Farhan   23     88
  8    Hari   22     80
  9    Isha   21     92
 11     Kim   20     83
 12    Liam   22     89
 14    Nina   20     86
 15      Om   23     91
 17   Quinn   21     84
 20    Tina   19     94
 21    Uday   21     81
 22   Veena   20     88
 24    Xena   21     85
 26    Zoya   20     87

----------------------------------------
Total Students: 25
High Scorers (>=80): 15
Failed Students (<40): 1
Average Score: 79.84
Topper: Eva with score 95

Execution Time: 0.066475 seconds


In [ ]:
import pandas as pd
import time
from multiprocessing import Pool, cpu_count

# Load CSV
df = pd.read_csv("students.csv", names=["ID", "Name", "Age", "Score"], header=None, skiprows=1)

# 1️⃣ Sequential Method
def sequential_analysis(df):
    start_time = time.time()

    high_scorers = df[df["Score"] >= 80]
    failed = df[df["Score"] < 40]
    total_students = len(df)
    average_score = df["Score"].mean()
    top_student = df[df["Score"] == df["Score"].max()]

    print("Sequential Processing")
    print("------------------------")
    print(high_scorers[["ID", "Name", "Age", "Score"]])
    print(f"\nTotal Students: {total_students}")
    print(f"High Scorers (>=80): {len(high_scorers)}")
    print(f"Failed Students (<40): {len(failed)}")
    print(f"Average Score: {average_score:.2f}")
    print(f"Topper: {top_student.iloc[0]['Name']} with score {top_student.iloc[0]['Score']}")

    end_time = time.time()
    print(f"Execution Time (Sequential): {end_time - start_time:.6f} seconds\n")


# 2️⃣ Parallel Processing using multiprocessing
def process_chunk(chunk):
    return {
        "high": chunk[chunk["Score"] >= 80],
        "fail": chunk[chunk["Score"] < 40],
        "sum": chunk["Score"].sum(),
        "count": len(chunk),
        "top": chunk.loc[chunk["Score"].idxmax()]
    }

def parallel_analysis(df):
    start_time = time.time()
    num_cores = cpu_count()
    chunks = np.array_split(df, num_cores)

    with Pool(num_cores) as pool:
        results = pool.map(process_chunk, chunks)

    high_scorers = pd.concat([r["high"] for r in results])
    failed = pd.concat([r["fail"] for r in results])
    total_students = sum([r["count"] for r in results])
    score_sum = sum([r["sum"] for r in results])
    average_score = score_sum / total_students
    top_student = max(results, key=lambda x: x["top"]["Score"])["top"]

    print("Parallel (Multiprocessing) Processing")
    print("-------------------------------------")
    print(high_scorers[["ID", "Name", "Age", "Score"]])
    print(f"\nTotal Students: {total_students}")
    print(f"High Scorers (>=80): {len(high_scorers)}")
    print(f"Failed Students (<40): {len(failed)}")
    print(f"Average Score: {average_score:.2f}")
    print(f"Topper: {top_student['Name']} with score {top_student['Score']}")

    end_time = time.time()
    print(f"Execution Time (Parallel): {end_time - start_time:.6f} seconds\n")

# Run both methods
sequential_analysis(df)
parallel_analysis(df)


Sequential Processing
------------------------
    ID     Name  Age  Score
1    3  Charlie   22     90
3    5      Eva   21     95
4    6   Farhan   23     88
6    8     Hari   22     80
7    9     Isha   21     92
9   11      Kim   20     83
10  12     Liam   22     89
12  14     Nina   20     86
13  15       Om   23     91
15  17    Quinn   21     84
18  20     Tina   19     94
19  21     Uday   21     81
20  22    Veena   20     88
22  24     Xena   21     85
24  26     Zoya   20     87

Total Students: 25
High Scorers (>=80): 15
Failed Students (<40): 1
Average Score: 79.84
Topper: Eva with score 95
Execution Time (Sequential): 0.005017 seconds



/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Parallel (Multiprocessing) Processing
-------------------------------------
    ID     Name  Age  Score
1    3  Charlie   22     90
3    5      Eva   21     95
4    6   Farhan   23     88
6    8     Hari   22     80
7    9     Isha   21     92
9   11      Kim   20     83
10  12     Liam   22     89
12  14     Nina   20     86
13  15       Om   23     91
15  17    Quinn   21     84
18  20     Tina   19     94
19  21     Uday   21     81
20  22    Veena   20     88
22  24     Xena   21     85
24  26     Zoya   20     87

Total Students: 25
High Scorers (>=80): 15
Failed Students (<40): 1
Average Score: 79.84
Topper: Eva with score 95
Execution Time (Parallel): 0.058889 seconds

